In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Browser Function Calling with the Vertex AI Gemini API & Python SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


In [ ]:
#upgrade the sdk for Gemini
# !pip3 install --upgrade --user google-cloud-aiplatform

In [2]:
# !pip install playwright > /dev/null
# !pip install  lxml

# If this is your first time using playwright, you'll have to install a browser executable.
# Running `playwright install` by default installs a chromium browser executable.
# !playwright install
# Also run this:
# !playwright install dependencies

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, it is recommended to restart the runtime. Run the following cell to restart the current kernel.

The restart process might take a minute or so.

In [3]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

#### We will be utilizing pre-built Langchain tools for the Google Search

____________

## Google Search
[Langchain Guide](https://python.langchain.com/docs/integrations/tools/google_search)

This notebook goes over how to use the google search component.

First, you need to set up the proper API keys and environment variables. To set it up, create the GOOGLE_API_KEY in the Google Cloud credential console (https://console.cloud.google.com/apis/credentials) and a GOOGLE_CSE_ID using the Programmable Search Engine (https://programmablesearchengine.google.com/controlpanel/create). Next, it is good to follow the instructions found here.

Then we will need to set some environment variables.

```python
import os

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""
```

In [2]:

import requests
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

from langchain.tools import Tool as LangchainTool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()


2023-12-15 22:11:38.397320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 22:11:40.508739: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-15 22:11:40.508978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or di

#### Create the Langchain tool for search

In [3]:
tool = LangchainTool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

In [4]:
tool.run("Obama's first name?")

"1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb ... OBAMA, II. Day. 4. 6b. Island. Year. 5b. Hour. 1961 7:24 P.M.. Oahu. 6d. Is Place\xa0... His last name, Obama, was derived from his Luo descent. Obama's parents met in ... First 100 days. Main article: First 100 days of Barack Obama's presidency. Apr 12, 2017 ... President Barack Obama's full name is Barack Hussein Obama. Does it mean that he is a Muslim? 199,232 Views. First Lady Michelle LaVaughn Robinson Obama is a lawyer, writer, and the wife of the 44th President, Barack Obama. She is the first African-American First\xa0... Apr 2, 2018 ... BARACK : Barkat and Mubarak both are derived from it in Hindi and Urdu. Roughly meaning blessing, abundance etc. Husen or Hussein from which\xa0... Barack Hussein Obama II was born August 4, 1961, in Honolulu, Hawaii, to parents Barack H. Obama, Sr., and Stanley Ann Dunham. His parents divorced when he\xa0... Jan 19, 2017 ... Hopeful parents named their sons for the

#### Get the tool parameters
Looking at the source code to find the parameters for the `.run` method for the tool


```python
def run(self, query: str) -> str:
        """Run query through GoogleSearch and parse result."""
        ...
```

In [14]:
search_func = FunctionDeclaration(
    name="google_search",
    description="Perform a Google Search",
    parameters={
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "Google Search Query"
        }
    }
},
)


You can then define a tool for the LLM to call that includes the `search_func`:

In [15]:
search_tool = Tool(
    function_declarations=[search_func],
)

### Use the Gemini Pro model

The Gemini Pro (`gemini-pro`) model is designed to handle natural language tasks, multiturn text and code chat, and code generation.

In [16]:
model = GenerativeModel("gemini-pro")

In [18]:
prompt = "What is the top news on Panera Bread?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[search_tool]
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "google_search"
        args {
          fields {
            key: "query"
            value {
              string_value: "Top news on Panera Bread"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 10
  total_token_count: 10
}

##### Quick inspection of one function call

In [19]:
response.candidates[0].content.parts[0].function_call

name: "google_search"
args {
  fields {
    key: "query"
    value {
      string_value: "Top news on Panera Bread"
    }
  }
}

In [20]:
from google.cloud.aiplatform_v1beta1.types.tool import FunctionCall
from vertexai.generative_models._generative_models import GenerationResponse
from typing import List

def use_search_tool(part: Part, search_tool: LangchainTool = tool):
    """
    Run a tool from the browser.

    Args:
        part: The part to run.
        tools_by_name: A dictionary of tool names to Langchain tools.

    Returns:
        A Part function response representing the results of the tool run.
    """

    single_function_call = part.function_call
    tool_name = single_function_call.name
    single_function_args = single_function_call.args
    if single_function_call.args is None:
        tool_params = {}
    else:
        tool_params = {k: single_function_args[k] for k in single_function_args}
    if tool_name != '':
        response = search_tool.run(tool_params)
        return(Part.from_function_response(
        name=tool_name,
        response={
            "content": {"response": response},
        }
        ))


def get_parts_from_response(response: GenerationResponse) -> List[FunctionCall]:
    """
    Get the parts from a generation response.

    Args:
        response: The generation response.

    Returns:
        A list of parts.
    """
    return(response.candidates[0].content)

#### We will now do a mult-turn chat
Example can be found here on [function calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-one-and-a-half-turn-curl-sample)

In [23]:
model = GenerativeModel("gemini-pro", 
                        generation_config={"temperature": 0},
                        tools=[search_tool])
chat = model.start_chat()

In [33]:
from pprint import pprint


# while_loop = True

# while while_loop:
response = chat.send_message("What is going on with Panera Bread Lemonaide?")
pprint(response)
function_calls = get_parts_from_response(response)
function_calls = function_calls.parts

# function_calls
tool_commands = [use_search_tool(fn_call) for fn_call in function_calls]
print("TOOL COMMAND: ", tool_commands)
if tool_commands[0] is None:
    # break
    pass
else:
    responses = [chat.send_message(command) for command in tool_commands]   

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "google_search"
        args {
          fields {
            key: "query"
            value {
              string_value: "What is going on with Panera Bread Lemonaide?"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 774
  total_token_count: 774
}

TOOL COMMAND:  [function_response {
  name: "google_search"
  response {
    fields {
      key: "content"
      value {
        struct_value {
          fields {
            key: "response"
            value {
   

In [43]:
pprint(responses[0].candidates[0].content.parts[0].text)

(" Panera Bread's Charged Lemonade has been linked to two deaths. The first "
 'death was a 77-year-old woman who died in September 2023 after drinking the '
 'lemonade. The second death was a 56-year-old man who died in December 2023 '
 'after drinking the lemonade. Both victims had pre-existing health '
 'conditions.\n'
 '\n'
 'The Charged Lemonade is a sparkling lemonade that contains caffeine and '
 'ginseng. It is marketed as a healthy alternative to soda. However, some '
 'experts have raised concerns about the safety of the drink. They say that '
 'the combination of caffeine and ginseng can be dangerous for people with '
 'heart problems or other health conditions.\n'
 '\n'
 'Panera Bread has defended the safety of the Charged Lemonade. The company '
 'says that the drink is safe for most people to consume. However, the company '
 'has warned people with heart problems or other health conditions to avoid '
 'drinking the lemonade.\n'
 '\n'
 'The FDA is currently investigating t